<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/ParticleSimulator1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import time
from IPython.display import clear_output

class Particle:
    def __init__(self, x, y, vx, vy, color):
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        self.color = color

    def move(self):
        self.x += self.vx
        self.y += self.vy

    def __str__(self):
        return f'{self.color}: x = {self.x}, y = {self.y}, vx = {self.vx}, vy = {self.vy}'

class Simulator:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.particles = []

    def add_particle(self, particle):
        self.particles.append(particle)

    def check_boundary_collision(self, p):
        if p.x <= 0 or p.x >= self.width - 1:
            p.vx = -p.vx
        if p.y <= 0 or p.y >= self.height - 1:
            p.vy = -p.vy

    def collide_particles(self, p1, p2):
        if abs(p1.x - p2.x) <= 1 and abs(p1.y - p2.y) <= 1:
            p1.vx, p2.vx = p2.vx, p1.vx
            p1.vy, p2.vy = p2.vy, p1.vy
            print(f'Столкновение {p1.color}, {p2.color}')

    def simulate_step(self):
        for p in self.particles:
            p.move()
            self.check_boundary_collision(p)

        for i in range(len(self.particles)):
            for j in range(i + 1, len(self.particles)):
                self.collide_particles(self.particles[i], self.particles[j])
                print(self.particles[i], self.particles[j])

def print_matrix(simulator):
    color_symbols = {
        'зеленый': '🟢',
        'синий': '🔵',
        'желтый': '🟡'
    }

    matrix = [['· ' for _ in range(simulator.height)] for _ in range(simulator.width)]
    for p in simulator.particles:
        x, y = int(p.x), int(p.y)
        if 0 <= x < simulator.width and 0 <= y < simulator.height:
            matrix[x][y] = color_symbols.get(p.color)

    for row in matrix:
        print(' '.join(row))

def main():
    sim = Simulator(10, 20)
    sim.add_particle(Particle(1, 8, 1, -1, 'зеленый'))
    sim.add_particle(Particle(5, 1, 0, 1, 'синий'))
    sim.add_particle(Particle(8, 1, -1, 1, 'желтый'))

    try:
        while True:
            clear_output(wait=True)
            print_matrix(sim)
            sim.simulate_step()
            time.sleep(0.1)
    except KeyboardInterrupt:
        print('Остановка')

if __name__ == "__main__":
    main()

·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  · 
·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  🔵 ·  ·  ·  ·  ·  ·  ·  · 
·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  · 
·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  · 
·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  · 
·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  🟢 · 
·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  🟡
·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  · 
·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  · 
·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  · 
зеленый: x = 4, y = 17, vx = -1, vy = -1 синий: x = 2, y = 12, vx = 1, vy = 1
зеленый: x = 4, y = 17, vx = -1, vy = -1 желтый: x = 6, y = 18, vx = 0, vy = -1
синий: x = 2, y = 12, vx = 1, vy = 1 желтый: x = 6, y = 18, vx = 0, vy = -1
Остановка


In [53]:
import time
import numpy as np
from IPython.display import clear_output
from scipy.spatial import KDTree
import random

class Particle:
    def __init__(self, x, y, vx, vy, color):
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        self.color = color

    def move(self):
        self.x += self.vx
        self.y += self.vy

class Simulator:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.particles = []
        self.collision_radius = 1.5

    def add_particle(self, particle):
        self.particles.append(particle)

    def create_rectangle(self, start_x, start_y, width, height, speed_range, color, density=1.0):
        particles = []

        for x in range(start_x, start_x + width):
            for y in range(start_y, start_y + height):
                if random.random() <= density:
                    vx = random.uniform(speed_range[0], speed_range[1])
                    vy = random.uniform(speed_range[0], speed_range[1])
                    particles.append(Particle(x, y, vx, vy, color))

        return particles


    def kdtree_collisions(self):
        positions = np.array([[p.x, p.y] for p in self.particles])
        tree = KDTree(positions)
        collision_pairs = tree.query_pairs(self.collision_radius)

        for i, j in collision_pairs:
            p1, p2 = self.particles[i], self.particles[j]
            distance = np.sqrt((p1.x - p2.x)**2 + (p1.y - p2.y)**2)
            if distance <= 1.0:
                p1.vx, p2.vx = p2.vx, p1.vx
                p1.vy, p2.vy = p2.vy, p1.vy

    def simulate_step(self):
        for p in self.particles:
            p.move()

        self.kdtree_collisions()

def print_matrix(simulator):
    color_symbols = {
        'красный': '🔴',
        'синий': '🔵'
    }

    matrix = [['· ' for _ in range(simulator.width)] for _ in range(simulator.height)]

    for p in simulator.particles:
        x, y = int(round(p.x)), int(round(p.y))
        if 0 <= x < simulator.width and 0 <= y < simulator.height:
            symbol = color_symbols.get(p.color)
            matrix[y][x] = symbol

    for row in matrix:
        print(''.join(row))

def main():
    width, height = 50, 50

    sim = Simulator(width, height)


    body_rect = sim.create_rectangle(
        start_x=20, start_y=20,
        width=10, height=10,
        speed_range=(-0.01, 0.01),
        color='синий',
        density=1
    )

    fire_rect = sim.create_rectangle(
        start_x=0, start_y=30,
        width=50, height=50,
        speed_range=(-0.5, 0.5),
        color='красный',
        density=0.8
    )

    for particle in fire_rect + body_rect:
        sim.add_particle(particle)

    try:
        for _ in range(1000):
            clear_output(wait=True)
            print_matrix(sim)
            sim.simulate_step()
            time.sleep(0.1)

    except KeyboardInterrupt:
        print('\nСимуляция остановлена')

if __name__ == "__main__":
    main()

· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · 
· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · 
· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · 
· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · 
· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · 
· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · 
· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · 
· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · 
· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · 
· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · ·